# Integrating tools/function with LLM using Azure OpenAI

In [1]:
!pip install openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [64]:
import os
import requests
import json
from openai import AzureOpenAI

In [87]:
api_key = "159466d48eaa43b3bb12ebf2d50e81fa"
api_version = "2023-07-01-preview" # "2023-05-15"
azure_endpoint = "https://anshuopenai.openai.azure.com/"
model_name = "gpt-35-turbo-16k"

In [88]:
from openai import AzureOpenAI

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
    api_version=api_version,
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=azure_endpoint,
    api_key = api_key,

)

In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. The Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code.

## Supported models
Not all model versions are trained with function calling data. Function calling is supported with the following models: gpt-4, gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-4-0613, gpt-3.5-turbo, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, and gpt-3.5-turbo-0613

In addition, parallel function calls is supported on the following models: gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-3.5-turbo-0125, and gpt-3.5-turbo-1106

In [89]:
# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [90]:
owm_api = "29af1cea50a401d8e624eea4660b3f59"

# create a dummy function to respond temperature
def get_current_weather2(location, unit="fahrenheit"):
  "This function can be used to fetch the current weather information when provided with details such as location and unit for temperature"

  url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={owm_api}"
  response = requests.get(url)
  temp = response.json()['main']['temp']
  forecast = [response.json()['weather'][0]['main'],response.json()['weather'][0]['description']]

  weather_info = {
      "location":location,
      "temperature":temp,
      "unit":'Kelvin',
      "forecast":forecast
  }
  return json.dumps(weather_info)

In [91]:
get_current_weather("Paris")

'{"location": "Paris", "temperature": "22", "unit": "fahrenheit"}'

In [92]:
get_current_weather2("Manila")

'{"location": "Manila", "temperature": 299.82, "unit": "Kelvin", "forecast": ["Clouds", "broken clouds"]}'

In [93]:
messages = [{"role":"user",'content':"what is AI?"}]
results = client.chat.completions.create(model = model_name, messages = messages)
results.choices[0].message

ChatCompletionMessage(content='AI stands for Artificial Intelligence. It refers to the simulation of human intelligence in machines that are programmed to think, learn, and problem-solve like humans. AI involves the development of advanced computer systems capable of performing tasks that would normally require human intelligence, such as visual perception, speech recognition, language translation, decision-making, and problem-solving. AI systems are designed to analyze large amounts of data, learn from patterns, and make predictions or suggestions based on that data. They can also adapt and improve their performance over time with experience. AI finds applications in various fields such as healthcare, finance, education, manufacturing, transportation, and more.', role='assistant', function_call=None, tool_calls=None)

In [97]:
tools = [{
                            "type":"function",
                            "function":{"name": "get_current_weather",
                           "description": "This function can be used to fetch the current weather information when provided with details such as location and unit for temperature",
                           "parameters": {
                               "type": "object",
                               "properties": {"location": {
                                   "type": "string",
                                   "description": "The name of city/state/country for which weather information is to be fetched",},},
                               "required": ["location",],},},}]



def get_response(messages, tools,model=model_name):
  response = client.chat.completions.create(model = model, messages = messages, tools=tools, tool_choice='auto', temperature=0.5,)
  response = response.choices[0].message
  tool_calls = response.tool_calls

  try:
    if tool_calls:
      print("Making a function call")
      # get available functions
      available_functions = {
          "get_current_weather":get_current_weather,
      }
      print(tool_calls)

      # get details to call the function assuming there is only one function call
      func_name = tool_calls[0].function.name
      func_to_call = available_functions[func_name]
      func_args = json.loads(tool_calls[0].function.arguments)

      # call the external api by calling the function
      func_response = func_to_call(**func_args)

      # again make a call to openai api to communicate results from external function/API
      messages.append(response)
      messages.append(
          {"tool_call_id":tool_calls[0].id,
          "role":"tool",
          "name":func_name,
          "content":str(func_response)}
      )

      second_response = client.chat.completions.create(
      model = "gpt-3.5-turbo-1106",
      messages = messages)
      return second_response
    else:
      return response.content
  except Exception as e:
      print("Error occurred ",e)
      return response




In [101]:
messages = [{"role": "user", "content": "Provide a 2 line explanation for AI"}]
response = get_response(messages, tools)
print(response)

AI stands for Artificial Intelligence. It is a field of computer science that focuses on creating intelligent machines that can perform tasks that typically require human intelligence, such as speech recognition, decision-making, problem-solving, and learning.


In [103]:
messages = [{"role": "user", "content": "How is the weather in Tokyo today?"}]
response = get_response(messages, tools)
print(response)

Making a function call
[ChatCompletionMessageToolCall(id='call_XD5sRUuZxT3v5RvmfJVTq3kU', function=Function(arguments='{\n  "location": "Tokyo"\n}', name='get_current_weather'), type='function')]
Error occurred  Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XD5sRUuZxT3v5RvmfJVTq3kU', function=Function(arguments='{\n  "location": "Tokyo"\n}', name='get_current_weather'), type='function')])
